In [2]:
import warnings
warnings.filterwarnings('ignore')
import pickle
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from gensim.models import FastText
import keras
import keras
from copy import deepcopy
import keras_contrib
from sklearn.metrics import f1_score,accuracy_score
from keras.models import load_model
from gensim.models import Word2Vec
import itertools
from more_itertools import locate
from keras.models import load_model
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_sample_weight

Using TensorFlow backend.


In [2]:
with open('train_paras.txt','r') as f:
    final=f.readlines()
    
for i in range(0,len(final)):
    final[i]=final[i][2:].split("', '")
    final[i][-1]=final[i][-1].replace("']\n",'')

In [3]:
final = final[500000:]

In [3]:
with open("tags.pkl", "rb") as fp:
    tags = pickle.load(fp)

In [5]:
tags = tags[500000:]

In [6]:
for i in range(0,len(tags)):
    tag=[]
    for j in range(0,len(tags[i])):
        x=tags[i][j].split('-')
        tag+=x
    tags[i]=tag

In [7]:
target = []
for i in range(0,len(final)):
    cvb = np.zeros(len(final[i]))
    for k in range(len(tags[i])):
        cvb[list(locate(final[i], lambda x: x == tags[i][k]))] =1
    target.append(list(cvb))

In [4]:
fasttext_model = FastText.load('fasttext_big_new_test.model')
w2v_model=Word2Vec.load('word2vec_big_new_test.model')

In [5]:
def transformation(x):
    for i in range(len(x)):
        x[i]=x[i][0:300]
        leng=300-len(x[i])
        x[i]=x[i]+leng*['pad']
        
    return x

In [6]:
def embed_fasttext(x):
    for i in range(len(x)):
        for j in range(300):
                try:
                    w2v = w2v_model.wv[x[i][j]]
                except:
                    w2v = np.zeros([150])
                try:
                    ft = fasttext_model.wv[x[i][j]]
                except:
                    ft = np.zeros([150])

                x[i][j] = np.concatenate([w2v,ft])

    
    return np.array(x)

In [7]:
input_text = keras.layers.Input(shape=(300,300))
lstm_1 = keras.layers.Bidirectional(keras.layers.CuDNNLSTM(units=128, return_sequences=True))(input_text)
drop = keras.layers.Dropout(0.3)(lstm_1)
dense = keras.layers.TimeDistributed(keras.layers.Dense(70, activation="relu"))(drop)
out=keras.layers.Dense(1,activation='sigmoid')(dense)

model = keras.Model(inputs = input_text,outputs=out)

In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 300, 300)          0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 300, 256)          440320    
_________________________________________________________________
dropout_1 (Dropout)          (None, 300, 256)          0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 300, 70)           17990     
_________________________________________________________________
dense_2 (Dense)              (None, 300, 1)            71        
Total params: 458,381
Trainable params: 458,381
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.compile(optimizer="adam", loss="binary_crossentropy",sample_weight_mode='temporal')

In [10]:
model = keras.models.load_model('joint_tags_model.h5')

In [11]:
def elmoembedding(x):
    x = transformation(x)
    y = embed_fasttext(deepcopy(x))
    return y

In [12]:
def target_processing(x):
    for i in range(len(x)):
        x[i] = x[i][0:300]
        x[i] = x[i] + [0.0] * (300 - len(x[i]))
    return np.array(x).reshape([len(x),300,1])

In [17]:
epochs = 0
while epochs < 7:    
    i = 0
    while i < len(final):
        fasttext = elmoembedding(deepcopy(final[i:i+64]))
        y_pass = target_processing(deepcopy(target[i:i+64]))
        
        i = i + 64
        sample_we = np.zeros([len(y_pass),300])
        for d in range(len(y_pass)):
            for v in range(len(y_pass[d])):
                if y_pass[d][v][0] == 0.0:
                    sample_we[d][v] = 1.0
                else:
                    sample_we[d][v] = 3.0
        
        a = model.train_on_batch(fasttext,y_pass,sample_weight=sample_we)
        
        if i % 32000 == 0:
            print(epochs,' ',i,a)
        
    epochs += 1

0   32000 0.06090642
0   64000 0.06387937
0   96000 0.055949654
0   128000 0.07063086
0   160000 0.0649589
0   192000 0.07127179
0   224000 0.0500456
0   256000 0.06429357
1   32000 0.061172027
1   64000 0.06385025
1   96000 0.05655585
1   128000 0.071429685
1   160000 0.059426174
1   192000 0.06678471
1   224000 0.050426688
1   256000 0.061554484
2   32000 0.060620848
2   64000 0.062835164
2   96000 0.057568587
2   128000 0.06546808
2   160000 0.059005164
2   192000 0.067020506
2   224000 0.048070457
2   256000 0.061298884
3   32000 0.057257697
3   64000 0.060488105
3   96000 0.05489551
3   128000 0.065703705
3   160000 0.056822218
3   192000 0.0637051
3   224000 0.049050763
3   256000 0.062495105
4   32000 0.057864152
4   64000 0.061800238
4   96000 0.053435884
4   128000 0.0670122
4   160000 0.05807192
4   192000 0.06479524
4   224000 0.048376452
4   256000 0.059959423
5   32000 0.057471544
5   64000 0.060350966
5   96000 0.054665845
5   128000 0.065186426
5   160000 0.058048733
5  

In [18]:
import gc
gc.collect()

100

In [21]:
model.save('joint_tags_model.h5')

In [13]:
from collections import Counter

In [40]:
vocab = list(itertools.chain(*tags))

In [41]:
c = Counter(vocab)

In [42]:
vocab = c.most_common()

In [43]:
vocab = [i[0] for i in vocab]

In [44]:
with open("test_paras.pkl", "rb") as fp:
    data = pickle.load(fp)

In [45]:
rty = np.zeros([1,300,1])
i = 0
while i < len(data):
    rty = np.append(rty,model.predict(elmoembedding(data[i:i+128])),axis= 0)
    i = i + 128
    if i % 1280 == 0:
        print(i)

1280
2560
3840
5120
6400
7680
8960
10240
11520
12800
14080
15360
16640
17920
19200
20480
21760
23040
24320
25600
26880
28160
29440
30720
32000
33280
34560
35840
37120
38400
39680
40960
42240
43520
44800
46080
47360
48640
49920
51200
52480
53760
55040
56320
57600
58880
60160
61440
62720
64000
65280
66560
67840
69120
70400
71680
72960
74240
75520
76800
78080
79360
80640
81920
83200
84480
85760
87040
88320
89600
90880
92160
93440
94720
96000
97280
98560
99840
101120
102400
103680
104960
106240
107520
108800
110080
111360
112640
113920
115200
116480
117760
119040
120320
121600
122880
124160
125440
126720
128000
129280
130560
131840
133120
134400
135680
136960
138240
139520
140800
142080
143360
144640
145920
147200
148480
149760
151040
152320
153600
154880
156160
157440
158720
160000
161280
162560
163840
165120
166400
167680
168960
170240
171520
172800
174080
175360
176640
177920
179200
180480
181760
183040
184320
185600
186880
188160
189440
190720
192000
193280


In [46]:
rty = rty[1:]

In [47]:
rty.shape

(194323, 300, 1)

In [48]:
rty = np.array(rty).reshape([len(data),300])

In [49]:
pred = []
for i in range(0,len(rty)):
    pred.append(list(set(np.array(data[i])[np.where(rty[i] > 0.4)[0]])))

In [50]:
pred[0:100]

[['ftl'],
 ['bash', 'phpunit', 'pear', 'lion', 'mountain', 'pecl'],
 ['menu', 'javascript', 'ajax', 'drop'],
 [],
 ['mysql', 'php', 'access', 'locking', 'database', 'phpmyadmin'],
 [],
 ['spreadsheet'],
 ['vsts', 'mediaelement', 'video', 'silverlight', '2008'],
 ['mobile', 'css', 'jquery', 'popup', 'scroll'],
 ['studio', 'maps', 'android', 'activity', 'google'],
 ['mobile', 'objectanimator', 'animation', 'android', 'rotation', 'google'],
 ['simplemodal', 'css', 'jquery'],
 ['swing', 'java'],
 ['animated', 'motion', 'background'],
 ['hashmap', 'parsing', 'java', 'split', 'string'],
 ['unix', 'io', 'leaks', 'select', 'perl', 'parallel'],
 ['html', 'charts', 'mysql', 'php', 'jquery', 'svg', 'bar', 'json'],
 ['spatial', 'graph', 'algorithm', 'scipy', 'knn', 'numpy'],
 ['prolog', 'algorithm'],
 ['facebook', 'database', 'iframe'],
 ['wpf', 'studio', 'xaml', '2015', 'windows', '10'],
 ['html',
  'grid',
  'css',
  'java',
  'jquery',
  'layout',
  'menu',
  'gridlayout',
  'javascript',
  'fl

In [51]:
bi_gram = [i for i in vocab if len(i.split('-')) == 2]

In [52]:
vocab = list(itertools.chain(*tags))
c = Counter(vocab)
vocab = c.most_common()
vocab = [i[0] for i in vocab]

In [53]:
tri_gram = [i for i in vocab if len(i.split('-')) == 3]

In [54]:
import itertools

In [55]:
new_pred_bigram = []
for k in range(0,len(pred)):
    temp = []
    x = [i[0]+'-'+i[1] for i in list(itertools.permutations(pred[k], 2))]
    for w in range(len(x)):
        if x[w] in bi_gram:
            temp.append(x[w])
    new_pred_bigram.append(temp)

In [56]:
new_pred_trigram = []
for k in range(0,len(pred)):
    temp = []
    x = [i[0]+'-'+i[1]+'-'+i[2] for i in list(itertools.permutations(pred[k], 3))]
    for w in range(len(x)):
        if x[w] in tri_gram:
            temp.append(x[w])
    new_pred_trigram.append(temp)

In [57]:
for i in range(len(new_pred_bigram)):
    temp = len(new_pred_bigram[i])
    j = 0
    if len(new_pred_bigram[i]) == 0 or len(new_pred_trigram[i]) == 0:
        continue
    while j < temp:
        temp1 = []
        for k in range(len(new_pred_trigram[i])):
            if set(new_pred_bigram[i][j].split('-')).issubset(set(new_pred_trigram[i][k].split('-'))):
                temp1.append(j)
        
        j = j +1
        
    temp1 = list(set(temp1))
    for index in sorted(temp1, reverse=True):
        del new_pred_bigram[i][index]

In [58]:
for i in range(len(new_pred_bigram)):
    new_pred_bigram[i] = new_pred_bigram[i] + new_pred_trigram[i]

In [59]:
with open('parrot.pkl', 'wb') as f:
    pickle.dump(new_pred_bigram, f)

In [60]:
new_pred_bigram[0:100]

[[],
 [],
 [],
 [],
 ['database-locking'],
 [],
 [],
 [],
 ['jquery-mobile-popup'],
 ['android-studio', 'android-maps', 'android-activity', 'google-maps'],
 ['android-animation'],
 [],
 [],
 [],
 ['string-parsing', 'string-split'],
 ['perl-select'],
 ['mysql-json', 'jquery-svg'],
 ['graph-algorithm'],
 [],
 ['facebook-iframe'],
 ['windows-10'],
 ['grid-layout', 'css-grid', 'jquery-layout', 'fluid-layout'],
 [],
 ['django-views', 'django-models'],
 [],
 [],
 ['spring-batch'],
 ['mysql-json', 'android-gridview', 'android-json'],
 [],
 [],
 [],
 ['spring-data',
  'spring-boot',
  'spring-mongodb',
  'mongodb-java',
  'spring-data-mongodb'],
 [],
 [],
 [],
 [],
 ['svn-repository'],
 [],
 ['database-cursor'],
 [],
 [],
 [],
 ['elastic-beanstalk', 'git-push', 'amazon-elastic-beanstalk'],
 [],
 [],
 [],
 [],
 ['watir-webdriver', 'selenium-webdriver', 'ruby-watir'],
 [],
 [],
 ['jquery-deferred'],
 ['sql-server-2008'],
 ['windows-phone', 'windows-controls'],
 ['android-layout', 'android-menu',